Fruits and Vegetables Classification


In [67]:
import os
import shutil
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import models
from tensorflow.keras import layers


In [68]:
ORIG_DATASET_DIR = "/Users/karlap/Documents/Universidad/dataset"
TARGET_DIR = "/Users/karlap/Documents/Universidad/dataset_dividido"

train_split = 0.70
val_split = 0.10
test_split = 0.20

random.seed(42)

for folder_name in os.listdir(ORIG_DATASET_DIR):
    folder_path = os.path.join(ORIG_DATASET_DIR, folder_name)
    if not os.path.isdir(folder_path):
        continue

    # Separar fruta y condición por el guion bajo _
    if '_' not in folder_name:
        continue

    fruit, condition = folder_name.split('_', 1)  # separa en dos partes
    images = [img for img in os.listdir(folder_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not images:
        continue
    
    random.shuffle(images)
    total = len(images)
    train_end = int(train_split * total)
    val_end = train_end + int(val_split * total)


    subsets = {
        'train': images[:train_end],
        'val': images[train_end:val_end],
        'test': images[val_end:]
    }

    for subset, files in subsets.items():
        dest_dir = os.path.join(TARGET_DIR, subset, fruit, condition)
        os.makedirs(dest_dir, exist_ok=True)
        for file in files:
            src_file = os.path.join(folder_path, file)
            dest_file = os.path.join(dest_dir, file)
            shutil.copy2(src_file, dest_file)

In [69]:
train_dir = os.path.join(TARGET_DIR, "train")
val_dir = os.path.join(TARGET_DIR, "val")
test_dir = os.path.join(TARGET_DIR, "test")

train_datagen = ImageDataGenerator (
    rescale = 1./255,
    rotation_range = 20, 
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    horizontal_flip = True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
	train_dir,
	target_size = (224, 224),
	batch_size = 32,
	class_mode ='categorical',
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 21719 images belonging to 14 classes.
Found 3501 images belonging to 14 classes.
Found 6803 images belonging to 14 classes.


Red neuronal convolutiva 


In [70]:
base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    input_shape=(224, 224, 3),
    pooling='avg',
    include_preprocessing=True
)

inputs = tf.keras.layers.Input(shape=(None, None, 3))
resized_inputs = tf.keras.layers.Resizing(224, 224)(inputs)
x = base_model(resized_inputs)
outputs = tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.AdamW(1e-4),
    metrics=['accuracy']
)

model.summary()

Model: "functional_38"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing_1 (Resizing)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 1280)           │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 14)             │        17,934 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,937,246 (22.65 MB)

 Trainable params: 5,876,638 (22.42 MB)

 Non-trainable params: 60,608 (236.75 KB)

In [71]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=2
)


Epoch 1/2
369/679 ━━━━━━━━━━━━━━━━━━━━ 42:58 8s/step - accuracy: 0.6269 - loss: 1.3406

KeyboardInterrupt: 